### Import Packages

In [1]:
import pandas as pd
import sqlite3

### Data Fetching

In [2]:
database = '/data/dialects_database.db'

In [ ]:
conn = sqlite3.connect(database)

In [22]:
cursor = conn.cursor()

In [23]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
print(tables)
conn.close()

[('id_text',), ('id_dialect',)]


In [28]:
def get_column_names(cursor):
    conn = sqlite3.connect(database)
    cursor = conn.cursor()

    cursor.execute("SELECT * FROM sqlite_master")

    column_names = [description[0] for description in cursor.description]

    conn.close()

    return column_names

print(f'The column names of sqlite_master are : {get_column_names(cursor)}')
cursor.close()

The column names of sqlite_master are : ['type', 'name', 'tbl_name', 'rootpage', 'sql']


In [34]:
def fetch_data(database: str) -> pd.DataFrame:
    """
    Connects to the SQLite database, fetches and merges data from the 'id_text' and 'id_dialect' tables.

    Args:
        database (str): The path to the SQLite database file.

    Returns:
        pd.DataFrame: The merged DataFrame containing data from 'id_text' and 'id_dialect' tables.
    """
    with sqlite3.connect(database) as conn:
        cursor = conn.cursor()

        tables = get_table_names(cursor)
        print_table_names(tables)

        df_raw = merge_tables(conn, 'id_text', 'id_dialect', 'id')

    return df_raw

def get_table_names(cursor: sqlite3.Cursor) -> list[str]:
    """
    Retrieves the names of all tables in the SQLite database.

    Args:
        cursor (sqlite3.Cursor): The database cursor.

    Returns:
        list[str]: A list of table names.
    """
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    return [row[0] for row in cursor.fetchall()]

def print_table_names(tables: list[str]) -> None:
    """
    Prints the names of all tables in the SQLite database.

    Args:
        tables (list[str]): A list of table names.
    """
    print("Tables:")
    for table in tables:
        print(table)

def merge_tables(conn: sqlite3.Connection, table1: str, table2: str, on_column: str) -> pd.DataFrame:
    """
    Merges two tables in the SQLite database on a specified column.

    Args:
        conn (sqlite3.Connection): The database connection.
        table1 (str): The name of the first table.
        table2 (str): The name of the second table.
        on_column (str): The column name on which to merge the tables.

    Returns:
        pd.DataFrame: The merged DataFrame.
    """
    table1_df = pd.read_sql_query(f'SELECT * FROM {table1}', conn)
    table2_df = pd.read_sql_query(f'SELECT * FROM {table2}', conn)
    merged_df = pd.merge(table1_df, table2_df, on=on_column)
    merged_df.drop(columns='id', inplace=True)
    return merged_df

In [36]:
df_raw = fetch_data(database)
df_raw

Tables:
id_text
id_dialect


,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...
147720,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل,SD
147721,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸,SD
147723,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان,SD


In [47]:
# save data as csv
df_raw.to_csv('/data/arabic_dialects.csv', index=False)

### Load data

In [49]:
# Attempt to read the CSV file with lineterminator specified
try:
    df = pd.read_csv('/data/arabic_dialects.csv', lineterminator='\n')
    print(df.head())
except pd.errors.ParserError as e:
    print(f"Error parsing CSV file: {e}")


                                                                                                                                                                                                                                                                                                       text  \
0                                                                                                                                                                                        @toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.   
1                                                                                                                                                                                                                                 @AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة   
2                                                              @smsm071990 @ALMOGRBE كل 20 

In [50]:
df

,text,dialect
0,@toha_Altomy @gy_yah قليلين ادب ومنافقين. لو اختهم او قريبتهم تتعاكس تقولي عليهم من نشاط حقوق المرأة من ردة فعلهم.,LY
1,@AlmFaisal 😂😂 الليبيين متقلبين!!!\nبس بالنسبة ليا انا ميليشياوي زمان وتوة,LY
2,@smsm071990 @ALMOGRBE كل 20 تانيه شاب ليبي بيرتاح لبنت مختلفة ويلاحظ انها غير كل البنات وبيحس كأنه يعرفها من زمان. بعدين يتزوج وحدة منهن وممكن اثنين ولاثلاثة وتنقلب الرومانسية لعياط وشياط وتهزيب\nand they live happily ever after\nذي اند,LY
3,@AboryPro @lyranoo85 رانيا عقليتك متخلفة. اولا الانسان يلي يحتاج اهل يخاف منهم علشان يكون محترم هو انسان قليل الادب اصلاً. ثانياً شن ذنب يلي معندهش اب ولا ام ولا خوت ولا خوات؟ يعني اليتيمة متستحقش تتزوج؟ وثالثاً ليش البنت هي بس لازم ادير الف حساب للراجل؟ هي متستحقش يندارلها الف حساب ولا هي عبدة؟,LY
4,@lyranoo85 شكلك متعقدة علشان الراجل لي تحبيه ازوج بنت يتيمة ولا بنت معندهش خوت. هدي اعصابك وفكينا من التخلف امتاعك,LY
...,...,...
147720,الناس دي بتنفخ في قربة مقدودة بالدارجي كده البلد دي لو ما الكيزان ديل راحو مافي حياه شريفه في البلد دي لان سبب الفساد والمصائب الموجودة #التطلع_لسودان_افضل,SD
147721,@Wail_Hamza @amiromer انت عايش وين بره السودان ولا شنو ماشايف البحصل دا,SD
147722,مااحرم نفسي ميسي حريف ولعاب برضو ..\nمدريدي وافتخر 🇪🇸,SD
147723,ياخي ديل ماخلو للشيطان وإبليس شي لكن يروحو وين من ربنا ونحن وراك وراك ياكوز ياعفن الأيام معدوده خلاص #تعرف_علي_الكيزان,SD
